In [1]:
#Import all necessary modules

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from osgeo import gdal
import matplotlib.pyplot as plt

In [4]:
import os
import numpy as np

In [5]:
from PIL import Image

In [6]:
from torch.utils.data import DataLoader
from torchvision import transforms

In [7]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from osgeo import gdal

###CREATING DATASET

In [13]:
#Use GPU if available to speed up the process
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
import pickle

#Load the images
with open('fire_images.pkl', 'rb') as f:
    fire_images = pickle.load(f)

with open('non_images.pkl', 'rb') as f:
    non_images = pickle.load(f)

In [16]:
#Assign labels to each image (1 if wildfire present, 0 if not) and store it as
# a single dataset, that will be used in training the classification model
class FireDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [17]:
#Creating labels for all the data: 1 for wildfire present, 0 for none
fire_labels = [1] * len(fire_images)
non_labels = [0] * len(non_images)

# Combine all images and labels (those with wildfires, and those without)
all_images = fire_images + non_images
all_labels = fire_labels + non_labels

# Shuffle the data
shuffled_indices = np.random.permutation(len(all_images))
shuffled_images = [all_images[i] for i in shuffled_indices]
shuffled_labels = [all_labels[i] for i in shuffled_indices]

#Transform the data to the required size and format to train under the ResNet model
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create the dataset
dataset = FireDataset(shuffled_images, shuffled_labels, transform=transform)


In [18]:
from torch.utils.data import random_split

#Splitting the test and train dataset in an 80:20 ratio
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size


train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [19]:
#Basic definitions
batch_size = 32
epochs = 10

In [20]:
#creating the dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

###TRAINING THE MODEL

In [22]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models


#Loading the ResNet-50 model
model = models.resnet50(pretrained=True)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
#Changing the last fully connected layer to suit our needs of binary classification
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=2)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
#Training the model on the images

for epoch in range(epochs):
    model.train() #Setting the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_fn(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    #Display loss of the model, as it progresses through training
    average_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {average_loss:.4f}")

print("Training complete")

In [23]:
#Function to save the model
def save_model(model, epoch, filename):
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(state, filename)

In [ ]:
file_name_model = "RESNET-50_DETECTINGWILDFIRE"
save_model(model, epoch, file_name_model)

###TEST THE MODEL


In [24]:
#Load the model
model.load_state_dict(torch.load("/content/drive/MyDrive/SAVED_MODELS/RESNET-50_DETECTINGWILDFIRE")["model_state_dict"])

<All keys matched successfully>

In [25]:
model.eval()  # Set the model to evaluation mode
correct_predictions = 0
total_samples = 0

with torch.no_grad():  #Disable gradient calculation during testing
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, dim=1)  # Get predicted class

        correct_predictions += (predicted == labels).sum().item() #Calculate the number of correct predictions
        total_samples += labels.size(0)  #Total samples = Batch size

#Calculate the accuracy and display it
accuracy = correct_predictions / total_samples
print(f"Accuracy on test data: {accuracy:.2%}")

Accuracy on test data: 100.00%


In [26]:
#Testing the model on the entire dataset
data_loader = DataLoader(dataset, batch_size=len(dataset), shuffle=True, num_workers=2, pin_memory=True)

In [27]:
model.eval()  # Set the model to evaluation mode
correct_predictions = 0
total_samples = 0

with torch.no_grad():  #Disable gradient calculation during testing
    for images, labels in data_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, dim=1)  # Get predicted class

        correct_predictions += (predicted == labels).sum().item() #Calculate the number of correct predictions
        total_samples += labels.size(0)  #Total samples = Batch size

#Calculate the accuracy and display it
accuracy = correct_predictions / total_samples
print(f"Accuracy on all data: {accuracy:.2%}")

Accuracy on all data: 100.00%
